In [4]:
%%capture
!pip install gensim
!pip install nmslib
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install
!pip install sentence_transformers
!pip install annoy
!pip install nltk
!pip install contractions
import contractions
import nmslib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import time
import re
import nltk
nltk.download('punkt')



sns.set()

In [5]:
data = pd.read_csv("94matched_url_body_casetitles.csv",usecols=["case_title_x","Description","body","case_title_y","url"])
# data.drop_duplicates(subset="case_title_x", inplace=True)
data = data.rename(columns={"case_title_y" : "post title","case_title_x":"case title","Description":"case description","body":"post body"})

In [6]:
# data[["case subject","post_titles"]]
data.drop_duplicates(inplace=True)

In [7]:
# JOIN COLUMNS TOGETHER 

data2 = pd.DataFrame()
data2["case_joined"] = data["case title"] + " " + data["case description"]
data2["post_joined"] = data["post title"] + " " + data["post body"]
data2

,case_joined,post_joined
0,[ARR] [Sev B] SR-2205300030001910 title: DBFS ...,DBFS API 2.0 Azure Azure Databricks Azure Azur...
1,[ARR] [Sev B] SR-2206010030001463 NoSuchMethod...,Databricks Runtime 10.4 LTS Docs Azure Azure D...
3,Customer is unable to run a jar job Here is on...,Databricks Runtime 10.4 LTS Docs Azure Azure D...
4,Databricks Enable Table access control has py4...,Access Azure Data Lake Storage using Azure Act...
6,Users can create databases poinint to ADLS gen...,Access Azure Data Lake Storage using Azure Act...
...,...,...
89,Databrick Cluster Issue Getting the error that...,Maximum execution context or notebook attachme...
90,[ARR] [Sev C] SR-2206210030001213-Policy issue...,Secret scopes Docs Azure Azure Databricks Docs...
91,Unable to save Git credentials on Git integrat...,GitHub version control Azure Azure Databricks ...
92,Delta table with partition unable to Handle In...,Table batch reads and writes Table batch reads...


In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stops = stopwords.words('english')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings('ignore')

stopwords_eng = stopwords.words('english')

# add datbricks stop words
# stopwords_eng +=["to","the","is","in","and","for","a","databricks","this","not","i","with","it","we","of","on","https","are","have","that","but","please","from","can","you","when","if","-","as","o","be","or","has","customer","see","using","an","time","any",".","our","t","help","hi","was","there","team","am","my","url","at","by","id","what","below","which","need","no","could","some","while","use","new","same","us","s","they","me","also","how","details","after","know","all","so","would","thanks","one","like","start","1","do","2","2022","however","00","up","about","hello","only","will","did","arr","your","true","tried","check","before","due","been","note","other","false","out","more","end","cannot","where","into","then","support","want","why","does","since","happened","were","found","last","details.","m","still","here","these","question","answer","time", "subscription", "subscriptions","didn","successfully","successful","went", "test", "tested","tests", "confirm", "confirmed","along","both","non","through","means","being","asked","ask","able","unable","believe","something","actually","back","identified","jan","feb","mar","apr","may","jun","jul","oct","nov","dec","chennai","kolkata","mumbai","delhi","never","worked","answers","stopped","occurring","issue","earlier","list","monday","onwards","already","exists","getting","error","fail","get","job","prod","utc-05","eastern","canada","additional","questions","problem","begin","00.000z","approximate","ongoing","leave","sun","mon","tue","wed","thu","fri","sat","utc-07","within","requesttypeid","subscriptiontype","location","resourcegroups","providers","utc-06","central","just","ultimately","because","problemstarttime","goes","having","said","its","locality","go","azureproductsubscriptionname","azureproductsubscriptionid","tenant","unifiedenterprise","supportplandisplayname","databricksconsent","grantpermission","hdinsightconsent","sharememorydump","following","during","resourceuri","workspaceid","workspaces","workspace","hence","caused","though","instanceid","getinstanceid","utc","seconds","elapsed","reason","recent","name","likely","most","times","must","specify","e.g","via","doesn","agent_additional_properties_do_not_edit","start","wasn","requestid","serverrequestid","yesterday","morning","needed","phone","contact","person","seamlessly","efficiently","yes","no","given","party","description","now","minutes","minute","instance","anything","feels","done","such","today","others","came","very","such","mins","avoid","http","https","fix","day","again","had","checked","check","even","sometimes","facing","asking","cause","necessary","steps","resolve","every","comes","under","keeps","showing","perfectly","personal","between","consent","insight","turned","off","email","hostedcontainer","hostprivateip","containerip","clusterid","resources","instantiatedresources","memorymb","ecus","cgroupshares","instancetype","metadata","containermetadata","happen","consider","org","sure","follow-up","jobname","bricks","own","involved","gets","directly","two","recently","happens","solve","issues","queries","possible","present","exist","path","call","deco", "raise","spark","pyspark","traceback","java.lang.thread.run"];

def remove_stops(query):
    new_word = []
    for word in query.split():
        if word in stopwords_eng or word.isdigit():
            pass
        else:
            new_word.append(word.strip().lower())
    
    return ' '.join(new_word)


from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()


def lemmatize(query):
    new_word = []
    for word in query.split():
        new_word.append(wordnet_lemmatizer.lemmatize(word, pos="v").strip())

    return ' '.join(new_word)


# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
 
ps = PorterStemmer()

def stem_(query):
    new_word = []
    for word in query.split():
        new_word.append(ps.stem(word).strip())

    return ' '.join(new_word)

In [18]:
data2["clean case"] = data2["case_joined"].apply(remove_stops)
# data["case title"] = data["case title"].apply(lemmatize)
data2["clean post"] = data2["post_joined"].apply(remove_stops)
# data["body"] = data["body"].apply(lemmatize)

In [10]:
# data2 = data[["case subject","post_titles","url"]]
data2

,case_joined,post_joined,clean case,clean post
0,[ARR] [Sev B] SR-2205300030001910 title: DBFS ...,DBFS API 2.0 Azure Azure Databricks Azure Azur...,[arr] [sev b] sr-2205300030001910 title: dbfs ...,dbfs api 2.0 azure azure databricks azure azur...
1,[ARR] [Sev B] SR-2206010030001463 NoSuchMethod...,Databricks Runtime 10.4 LTS Docs Azure Azure D...,[arr] [sev b] sr-2206010030001463 nosuchmethod...,databricks runtime 10.4 lts docs azure azure d...
3,Customer is unable to run a jar job Here is on...,Databricks Runtime 10.4 LTS Docs Azure Azure D...,customer unable run jar job here one latest ru...,databricks runtime 10.4 lts docs azure azure d...
4,Databricks Enable Table access control has py4...,Access Azure Data Lake Storage using Azure Act...,databricks enable table access control py4j.se...,access azure data lake storage using azure act...
6,Users can create databases poinint to ADLS gen...,Access Azure Data Lake Storage using Azure Act...,users create databases poinint adls gen2 stora...,access azure data lake storage using azure act...
...,...,...,...,...
89,Databrick Cluster Issue Getting the error that...,Maximum execution context or notebook attachme...,databrick cluster issue getting error cluster ...,maximum execution context notebook attachment ...
90,[ARR] [Sev C] SR-2206210030001213-Policy issue...,Secret scopes Docs Azure Azure Databricks Docs...,[arr] [sev c] sr-2206210030001213-policy issue...,secret scopes docs azure azure databricks docs...
91,Unable to save Git credentials on Git integrat...,GitHub version control Azure Azure Databricks ...,unable save git credentials git integration ta...,github version control azure azure databricks ...
92,Delta table with partition unable to Handle In...,Table batch reads and writes Table batch reads...,delta table partition unable handle insert ove...,table batch reads writes table batch reads wri...


In [19]:
import re
nltk.download('averaged_perceptron_tagger')

True

In [20]:
def clean(query):
  q = re.sub(r'[^\w]', ' ', query)
  q = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", q)
  
  # keep only nouns and proper nouns
  new_word = []
  for word in nltk.word_tokenize(q):
    tagged = nltk.pos_tag([word])
    tag = tagged[0][1]
    if tag == "NN" or tag == "NNP":
      new_word.append(word)
  
  return ' '.join(new_word)

In [21]:
data2["clean case"] = data2["clean case"].apply(clean)
# data["case title"] = data["case title"].apply(lemmatize)
data2["clean post"] = data2["clean post"].apply(clean)


In [22]:
data2

,case_joined,post_joined,clean case,clean post
0,[ARR] [Sev B] SR-2205300030001910 title: DBFS ...,DBFS API 2.0 Azure Azure Databricks Azure Azur...,arr sev b sr title dbfs api mb upload limitati...,dbfs api azure azure azure azure read table re...
1,[ARR] [Sev B] SR-2206010030001463 NoSuchMethod...,Databricks Runtime 10.4 LTS Docs Azure Azure D...,arr sev b sr nosuchmethoderrorcomeswhileinstal...,runtime docs azure azure docs azure azure read...
3,Customer is unable to run a jar job Here is on...,Databricks Runtime 10.4 LTS Docs Azure Azure D...,customer jar job issue https adb net o job cus...,runtime docs azure azure docs azure azure read...
4,Databricks Enable Table access control has py4...,Access Azure Data Lake Storage using Azure Act...,table access control py4j security py4jsecurit...,access azure lake storage azure directory cred...
6,Users can create databases poinint to ADLS gen...,Access Azure Data Lake Storage using Azure Act...,create poinint adls gen2 storage container wor...,access azure lake storage azure directory cred...
...,...,...,...,...
89,Databrick Cluster Issue Getting the error that...,Maximum execution context or notebook attachme...,databrick cluster issue error cluster driver c...,maximum execution context notebook attachment ...
90,[ARR] [Sev C] SR-2206210030001213-Policy issue...,Secret scopes Docs Azure Azure Databricks Docs...,arr sev sr policy issue cluster issue customer...,secret docs azure azure docs azure azure read ...
91,Unable to save Git credentials on Git integrat...,GitHub version control Azure Azure Databricks ...,git git integration tab git git integration ta...,github version control azure azure azure azure...
92,Delta table with partition unable to Handle In...,Table batch reads and writes Table batch reads...,delta table partition handle insert overwite t...,table batch table batch delta lake apache spar...


In [23]:
from sentence_transformers import SentenceTransformer

In [17]:
# sample = data2.sample(2)
sample = data2.copy()

In [18]:
sample

,case_joined,post_joined,clean case,clean post
0,[ARR] [Sev B] SR-2205300030001910 title: DBFS ...,DBFS API 2.0 Azure Azure Databricks Azure Azur...,arr sev b sr title dbfs api mb upload limitati...,dbfs api azure azure azure azure read table re...
1,[ARR] [Sev B] SR-2206010030001463 NoSuchMethod...,Databricks Runtime 10.4 LTS Docs Azure Azure D...,arr sev b sr nosuchmethoderrorcomeswhileinstal...,runtime docs azure azure docs azure azure read...
3,Customer is unable to run a jar job Here is on...,Databricks Runtime 10.4 LTS Docs Azure Azure D...,customer jar job issue https adb net o job cus...,runtime docs azure azure docs azure azure read...
4,Databricks Enable Table access control has py4...,Access Azure Data Lake Storage using Azure Act...,table access control py4j security py4jsecurit...,access azure lake storage azure directory cred...
6,Users can create databases poinint to ADLS gen...,Access Azure Data Lake Storage using Azure Act...,create poinint adls gen2 storage container wor...,access azure lake storage azure directory cred...
...,...,...,...,...
89,Databrick Cluster Issue Getting the error that...,Maximum execution context or notebook attachme...,databrick cluster issue error cluster driver c...,maximum execution context notebook attachment ...
90,[ARR] [Sev C] SR-2206210030001213-Policy issue...,Secret scopes Docs Azure Azure Databricks Docs...,arr sev sr policy issue cluster issue customer...,secret docs azure azure docs azure azure read ...
91,Unable to save Git credentials on Git integrat...,GitHub version control Azure Azure Databricks ...,git git integration tab git git integration ta...,github version control azure azure azure azure...
92,Delta table with partition unable to Handle In...,Table batch reads and writes Table batch reads...,delta table partition handle insert overwite t...,table batch table batch delta lake apache spar...


In [ ]:
# def tokenize_into_words(query):
  # return word_tokenize(query)

In [19]:
sample.iloc[1]["case_joined"]

'[ARR] [Sev B] SR-2206010030001463 NoSuchMethoderrorComesWhileinstallingdifferent libraries on databricks cluster [Issue]\r\n No such method erorr for installed libs.\r\nthe customer need to know the reason why this error happens and how to solve it.\r\n\r\nError\r\nCaused by: NoSuchMethodError: reactor.netty.http.client.HttpClient.resolver(Lio/netty/resolver/AddressResolverGroup;)Lreactor/netty/transport/ClientTransport;\r\n\r\n-New problem, worked before\r\n\r\nWhen this issue started:\r\nWed, Jun 1, 2022, 12:00 AM (UTC+05:30) Chennai, Kolkata, Mumbai, New Delhi\r\n\r\nNotebook URL\r\nhttps://adb-4359460891691173.13.azuredatabricks.net/?o=4359460891691173#notebook/2152658368282677/command/4361637730630314\r\n\r\nCluster URL:\r\nhttps://adb-4359460891691173.13.azuredatabricks.net/?o=4359460891691173#setting/clusters/1026-114902-flue504/configuration\r\n\r\n[What I checked]\r\nI found in previous case there are similar errors happening on DBR 10.X.\r\nActually I found init script as wo

In [ ]:
# the unique words are too many, most of them are useless
# we can use TF-IDF to get the most important words only & then find synonyms amonst those words

# np.unique(sample["token_post"].values)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,1),min_df=1)


In [21]:
def show_tfidf_values(doc):
  # print(doc)
  results = tfidf.fit_transform([doc])

  # these are the unique words and their numerical values(encodings) that are assigned to them
  # print(tfidf.vocabulary_,"\n")

  # get feature names according to their numerical encoded values (sorted from low to high)
  feature_names = tfidf.get_feature_names()

  # these are the tf-idf values for each uniue word
  # 0 means the (0th) first document or row, 1 means the (1st) second document or row 
  # (0,10) ===== 0.199 means in the first row, the word with vector 10 has 0.199 tf-idf value 
  # print(results)

  first_doc_results = results[0]

  # get encoded values for each title
  vectorized_cols = results.nonzero()[1]
  vectorized_cols

  combined_scores = results.toarray()[0] 
  combined_scores

  # formatting the data to create tables/df
  words = []
  vectors = []
  tfidfs = []

  # print(f'{"word(s)":<25} {"encoding":<10} {"tf-idf":<10}\n')
  for vector in vectorized_cols:
    for word,vect in tfidf.vocabulary_.items():
      if vect == vector and combined_scores[vector] > 0.1:
        words.append(word)
        vectors.append(vector)
        tfidfs.append(combined_scores[vector])
        # print(f'{word:<25} {vector:<10} {combined_scores[vector]:<10}')
  
  # df = pd.DataFrame({
  #   "word" : words,
  #   "encoded word" : vectors,
  #   "tf-idf" : tfidfs
  # })

  # df = df.sort_values(by="tf-idf",ascending=False)
  return words
  # return df
              

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
model1 = SentenceTransformer('whaleloops/phrase-bert')
# model1 = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/670 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [29]:
def helperFunction(cases,posts):
  overall = []
  for c, p in zip(cases,posts):
    case_top_words = show_tfidf_values(c)
    post_top_words = show_tfidf_values(p)
    # print(case_top_words,post_top_words)
    encoding_case = model1.encode(case_top_words)
    encoding_post = model1.encode(post_top_words)

    main = []
    for i,c in enumerate(encoding_case):
      temp = []
      for j,p in enumerate(encoding_post):
        score = cosine_similarity(np.array(c).reshape(1,-1),np.array(p).reshape(1,-1))
        if score[0][0] > 0.4 and (case_top_words[i] != post_top_words[j]):
          print(case_top_words[i], post_top_words[j])
          try:
            int(case_top_words[i])
            int(post_top_words[j])
          except Exception as e:
            temp.append((case_top_words[i], post_top_words[j], score[0][0]))
      
      if len(temp):
        main.append(temp)

    overall.append(main)
    print(">>>>>>>>>>>>>>>>>>>>")
  return overall



In [30]:
synonyms = helperFunction(sample["clean case"].values, sample["clean post"].values)

Streaming output truncated to the last 5000 lines.
message group
message security
message workspace
message vnet
message network
message azure
error virtualnetwork
error ip
error subnet
error cluster
error use
error group
error security
error workspace
error vnet
error network
error azure
pipeline virtualnetwork
pipeline ip
pipeline subnet
pipeline cluster
pipeline group
pipeline security
pipeline workspace
pipeline vnet
pipeline network
pipeline azure
sdk virtualnetwork
sdk ip
sdk subnet
sdk cluster
sdk use
sdk group
sdk security
sdk workspace
sdk create
sdk vnet
sdk network
sdk azure
ml virtualnetwork
ml ip
ml subnet
ml cluster
ml group
ml security
ml workspace
ml vnet
ml azure
azure virtualnetwork
azure ip
azure subnet
azure cluster
azure group
azure security
azure workspace
azure vnet
azure network
version virtualnetwork
version ip
version subnet
version cluster
version use
version group
version security
version workspace
version create
version vnet
version network
version azure
ns

In [31]:
data_new = pd.DataFrame()
data_new["case title"]  = data["case title"]
data_new["post title"]  = data["post title"]
data_new["synonyms (case->post)"]  = synonyms
data_new["url"]  = data["url"]


In [32]:
data_new.head()

,case title,post title,synonyms (case->post),url
0,[ARR] [Sev B] SR-2205300030001910 title: DBFS ...,DBFS API 2.0,"[[(result, path, 0.5178842), (result, director...",https://docs.microsoft.com/en-us/azure/databri...
1,[ARR] [Sev B] SR-2206010030001463 NoSuchMethod...,Databricks Runtime 10.4 LTS,"[[(maven2, jetty, 0.5363725), (maven2, org, 0....",https://docs.microsoft.com/en-us/azure/databri...
3,Customer is unable to run a jar job,Databricks Runtime 10.4 LTS,"[[(line, jetty, 0.52333957), (line, org, 0.485...",https://docs.microsoft.com/en-us/azure/databri...
4,Databricks Enable Table access control has py4...,Access Azure Data Lake Storage using Azure Act...,"[[(fan, spark, 0.5712142), (fan, name, 0.49995...",https://docs.microsoft.com/en-us/azure/databri...
6,Users can create databases poinint to ADLS gen...,Access Azure Data Lake Storage using Azure Act...,"[[(spark, account, 0.40055662), (spark, gen2, ...",https://docs.microsoft.com/en-us/azure/databri...


In [33]:
data_new.to_csv("synonym_test2.csv")

In [23]:
data.iloc[38]["case description"]

"Cx shared a screenshot(attached) from AWS Databricks service that they can find daily detail usage report and asked if there is a view to monitor daily DBU usage on each workspace in Azure as well. I think Azure doesn't provide a daily usage report on DBU.\r\n\r\nThe background of asking this question is that customer is using more than 1000 clusters and they are planning to purchase the resource in DBCU (Databricks Commit Unit) in advance, not paying after using the resource. Therefore, they would like to understand the current DBU usage, so that they can estimate how much they need to purchase. I think we can calculate the DBU by checking each DBU/hour from the cluster and multiplying the used time, but since there are too many clusters, it's not an easy situation. Can you help identify the daily dbu usage on customers' every workspace and cluster?\r\n\r\n\r\nFYI. customer can check the daily cost(billing) usage by referencing the following link, but no DBU usage view is provided.\r

In [30]:
# # tf-idf score for 1 cleaned case (most important terms for the case)




# case_top_words = show_tfidf_values(sample["clean case"].values[0])["word"].to_list()

In [31]:
# # tf-idf score for 1 cleaned case (most important terms for the case)

# post_top_words = show_tfidf_values(sample["clean post"].values[0])["word"].to_list()

In [33]:
# case_top_words

In [34]:
# post_top_words

In [ ]:
# we need to convert each of these words into a vector

model1 = SentenceTransformer('all-mpnet-base-v2')
# model2 = SentenceTransformer('sentence-transformers/msmarco-distilroberta-base-v2')
# model3 = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
model4 = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')


encoding_case1 = model1.encode(case_top_words)
encoding_post1 = model1.encode(post_top_words)

# encoding_case2 = model2.encode(case_top_words)
# encoding_post2 = model2.encode(post_top_words)

# encoding_case3 = model3.encode(case_top_words)
# encoding_post3 = model3.encode(post_top_words)

# encoding_case4 = model4.encode(case_top_words)
# encoding_post4 = model4.encode(post_top_words)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
main = []
for i,c in enumerate(encoding_case1):
  temp = []
  for j,p in enumerate(encoding_post1):
    score = cosine_similarity(np.array(c).reshape(1,-1),np.array(p).reshape(1,-1))
    if score[0][0] > 0.3 and (case_top_words[i] != post_top_words[j]):
      # print(f"case - {case_top_words[i]:<15} | post - {post_top_words[j]:<20} | score - {score[0][0]:<10}")
      temp.append((case_top_words[i], post_top_words[j], score[0][0]))
  
  # print(temp,"====")
  if len(temp):
    main.append(temp)

print(main)

[[('key', 'vault', 0.30340114), ('key', 'keys', 0.839638), ('key', 'values', 0.3424498), ('key', 'encryption', 0.3588385), ('key', 'keyvault', 0.4879415)], [('vault', 'key', 0.30340114), ('vault', 'azure', 0.42990232), ('vault', 'workspace', 0.34053826), ('vault', 'databricks', 0.3453194), ('vault', 'encryption', 0.42931822), ('vault', 'dbfs', 0.3580438), ('vault', 'keyvault', 0.38726026)], [('azure', 'vault', 0.42990232), ('azure', 'az', 0.37739277), ('azure', 'encryption', 0.31328765)], [('caused', 'create', 0.3545258)], [('by', 'managed', 0.31131506)], [('job', 'key', 0.32800734), ('job', 'name', 0.33898902), ('job', 'workspace', 0.308648), ('job', 'managed', 0.3438043), ('job', 'customer', 0.35924745)], [('failed', 'managed', 0.33484268)], [('scope', 'vault', 0.31902128)], [('details', 'name', 0.31834835), ('details', 'customer', 0.32441548)], [('eastus2', 'name', 0.36203197)], [('error', 'customer', 0.30778134)], [('issue', 'key', 0.3551766), ('issue', 'name', 0.33629715), ('issue

In [ ]:
for i,c in enumerate(encoding_case4):
  for j,p in enumerate(encoding_post4):
    score = cosine_similarity(np.array(c).reshape(1,-1),np.array(p).reshape(1,-1))
    if score[0][0] > 0.4:
      print(f"case - {case_top_words[i]:<15} | post - {post_top_words[j]:<20} | score - {score[0][0]:<10}")
    
  print("-------------------")

case - delta           | post - delta                | score - 1.0       
case - delta           | post - azure                | score - 0.40995556116104126
-------------------
case - 00              | post - spark                | score - 0.44311681389808655
case - 00              | post - azure                | score - 0.5298174619674683
case - 00              | post - threshold            | score - 0.4500187635421753
case - 00              | post - the                  | score - 0.5314618945121765
case - 00              | post - hours                | score - 0.45314908027648926
-------------------
-------------------
case - files           | post - files                | score - 1.0       
-------------------
-------------------
case - time            | post - hours                | score - 0.5202933549880981
-------------------
case - simply          | post - azure                | score - 0.41517174243927
case - simply          | post - the                  | score - 0.4933063387